In [3]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from math import sqrt, exp, ceil
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [4]:
# https://jovian.ai/aakashns/02-linear-regression

# load house price related data
# https://scikit-learn.org/stable/modules/preprocessing.html


data=pd.read_excel('16實證資料.xlsx', usecols='A:Y, AA:AG, AI:AJ, AN:AO')
min_max_scaler = preprocessing.MinMaxScaler()

data['台鐵_直距(公里)'] = min_max_scaler.fit_transform(np.array(data['台鐵_直距(公里)']).reshape(-1, 1))
data['百貨_直距(公里)'] = min_max_scaler.fit_transform(np.array(data['百貨_直距(公里)']).reshape(-1, 1))
data['大專_直距(公里)'] = min_max_scaler.fit_transform(np.array(data['大專_直距(公里)']).reshape(-1, 1))
data['捷運_路距(公里)'] = min_max_scaler.fit_transform(np.array(data['捷運_路距(公里)']).reshape(-1, 1))
data['公園_路距(公里)'] = min_max_scaler.fit_transform(np.array(data['公園_路距(公里)']).reshape(-1, 1))
data['公車站_路距(公里)'] = min_max_scaler.fit_transform(np.array(data['公車站_路距(公里)']).reshape(-1, 1))
data['國小_路距(公里)'] = min_max_scaler.fit_transform(np.array(data['國小_路距(公里)']).reshape(-1, 1))
data['國中_路距(公里)'] = min_max_scaler.fit_transform(np.array(data['國中_路距(公里)']).reshape(-1, 1))
data['高中職_路距(公里)'] = min_max_scaler.fit_transform(np.array(data['高中職_路距(公里)']).reshape(-1, 1))


# data clean and transform 
for d in data['樓層']:
    if type(d)!=int:
        data.drop(data.loc[data['樓層']==d].index, inplace=True)
        

print(data.head(), data.columns)


     LN(總價)  中正區  大安區  松山區  南港區  內湖區  信義區  中山區  士林區  文山區  ...  公園_路距(公里)  \
0  7.681560    0    0    0    0    1    0    0    0    0  ...   0.031904   
1  7.299797    0    0    0    0    0    0    0    0    0  ...   0.047884   
2  7.244228    0    1    0    0    0    0    0    0    0  ...   0.077494   
3  7.494430    0    0    0    0    0    0    1    0    0  ...   0.052085   
4  7.937375    0    0    0    0    0    0    0    1    0  ...   0.087862   

   公車站_路距(公里)  國小_路距(公里)  國中_路距(公里)  高中職_路距(公里)  採光  管理  邊間  裝潢  景觀  
0    0.177010   0.371121   0.551265    0.441955   0   0   1   1   0  
1    0.001689   0.142749   0.336884    0.558161   1   0   0   0   0  
2    0.097310   0.105114   0.126316    0.105385   0   1   0   1   0  
3    0.035048   0.111924   0.104736    0.137254   1   0   0   0   0  
4    0.083993   0.061999   0.304394    0.165038   0   0   0   0   0  

[5 rows x 36 columns] Index(['LN(總價)', '中正區', '大安區', '松山區', '南港區', '內湖區', '信義區', '中山區', '士林區', '文山區',
       '北投區', '大同區',

In [5]:

X=torch.from_numpy(np.array(data.drop(columns='LN(總價)'), dtype='float32'))
y=torch.from_numpy(np.array(data['LN(總價)'], dtype='float32'))

# 80%, 20%
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    random_state=12
)


In [6]:
# Define dataset
train_ds = TensorDataset(X_train, y_train)
test_ds = TensorDataset(X_test, y_test)

# Define data loader
batch_size = 5
train_dl = DataLoader(train_ds, batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size, shuffle=True)

# Define model
input_size = list(X_train.shape)[1]
model = nn.Linear(input_size, 1) #1234
print(model.weight)
print(model.bias)


# # Function to save the model 
def saveModel(): 
    model_scripted = torch.jit.script(model) # Export to TorchScript
    model_scripted.save('model_final.pt') # Save



Parameter containing:
tensor([[-0.1260,  0.1328,  0.1010,  0.0568, -0.0545, -0.1444,  0.0280,  0.0829,
          0.1054,  0.0941,  0.1536, -0.0038,  0.0698,  0.0599,  0.1208, -0.1020,
         -0.0247,  0.0375, -0.1687, -0.0934,  0.1346, -0.0377, -0.0670,  0.0455,
         -0.0520, -0.1260,  0.0645, -0.1435,  0.0608, -0.0736,  0.0930,  0.1436,
          0.1211, -0.0394, -0.1511]], requires_grad=True)
Parameter containing:
tensor([-0.0292], requires_grad=True)


In [7]:
# Generate predictions
preds = model(X_train)
preds


# Define loss function
loss_fn = F.mse_loss

loss = loss_fn(model(X_train).ravel(), y_train)
print(loss)

# Define optimizer
opt = torch.optim.Adam(model.parameters(), lr=1e-5)

tensor(72.1154, grad_fn=<MseLossBackward0>)


In [8]:
# Utility function to train the model
def fit(num_epochs, model, loss_fn, opt, train_dl):
    
    # Repeat for given number of epochs
    for epoch in range(num_epochs):
        running_train_loss = 0.0
        
        # Train with batches of data
        for xb,yb in train_dl:
            
            # 1. Generate predictions
            pred = model(xb).ravel()
            #print(pred)
            #print(yb)
            
            # 2. Calculate loss
            loss = loss_fn(pred, yb)
            
            # 3. Compute gradients
            loss.backward()
            
            # 4. Update parameters using gradients
            opt.step()
            
            # 5. Reset the gradients to zero
            opt.zero_grad()
            
            # Calculate training loss value 
            running_train_loss += loss.item()
        
        train_loss_value = running_train_loss/len(train_dl) 
        
        # Print the progress
        if (epoch+1) % 10 == 0:
            print('Epoch [{}/{}], Training Loss: {:.4f}%'.format(epoch+1, num_epochs, train_loss_value))
    

fit(1000, model, loss_fn, opt, train_dl)
saveModel()



Epoch [10/1000], Training Loss: 59.3074%
Epoch [20/1000], Training Loss: 47.3226%
Epoch [30/1000], Training Loss: 36.7603%
Epoch [40/1000], Training Loss: 27.6405%
Epoch [50/1000], Training Loss: 19.9284%
Epoch [60/1000], Training Loss: 13.6432%
Epoch [70/1000], Training Loss: 8.7672%
Epoch [80/1000], Training Loss: 5.2938%
Epoch [90/1000], Training Loss: 3.1828%
Epoch [100/1000], Training Loss: 2.1896%
Epoch [110/1000], Training Loss: 1.8023%
Epoch [120/1000], Training Loss: 1.6217%
Epoch [130/1000], Training Loss: 1.4976%
Epoch [140/1000], Training Loss: 1.3950%
Epoch [150/1000], Training Loss: 1.3049%
Epoch [160/1000], Training Loss: 1.2220%
Epoch [170/1000], Training Loss: 1.1479%
Epoch [180/1000], Training Loss: 1.0814%
Epoch [190/1000], Training Loss: 1.0218%
Epoch [200/1000], Training Loss: 0.9657%
Epoch [210/1000], Training Loss: 0.9124%
Epoch [220/1000], Training Loss: 0.8654%
Epoch [230/1000], Training Loss: 0.8207%
Epoch [240/1000], Training Loss: 0.7799%
Epoch [250/1000], T

In [9]:
# test data
# https://medium.com/analytics-vidhya/linear-regression-with-pytorch-147fed55f138

# model results
# 衡量線性迴歸的MSE 、 RMSE、 MAE、r2


model = torch.jit.load('model_final.pt')
model.eval()

y_pred_test = model(X_test)

y_pred_test = [y_pred_test[x].item() for x in range(len(y_pred_test))]
# Comparing Actual and predicted values
df = {}
df['Actual Observation'] = y_test
df['Predicted Price'] = y_pred_test


for i in range(len(y_test)):
    df['Actual Observation'][i] = exp(df['Actual Observation'][i])
    df['Predicted Price'][i] = exp(df['Predicted Price'][i])
df = pd.DataFrame(df)
print(df.head())


print('Price')
print("mean_absolute_error:",mean_absolute_error(df['Actual Observation'],df['Predicted Price']))
print("mean_squared_error:",mean_squared_error(df['Actual Observation'],df['Predicted Price']))
print("rmse:",sqrt(mean_squared_error(df['Actual Observation'],df['Predicted Price'])))
print("r2 score:",r2_score(df['Actual Observation'],df['Predicted Price']))



  Actual Observation  Predicted Price
0  tensor(4599.9980)      4376.166094
1  tensor(8800.0010)      7038.349334
2  tensor(1038.0001)       927.093236
3  tensor(1100.0001)       722.768081
4  tensor(9687.9990)      9105.052679
Price
mean_absolute_error: 716.9258672389066
mean_squared_error: 2212179.665504863
rmse: 1487.3397949039295
r2 score: 0.8452700366531083
